## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embeddings, load one from gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [ ]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

Unknown instance spec: Please select VM configuration

In [2]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

"What TV shows or books help you read people's body language?\n"

In [3]:
data[:10]

["Can I get back with my ex even though she is pregnant with another guy's baby?\n",
 'What are some ways to overcome a fast food addiction?\n',
 'Who were the great Chinese soldiers and leaders who fought in WW2?\n',
 'What are ZIP codes in the Bay Area?\n',
 'Why was George RR Martin critical of JK Rowling after losing the Hugo award?\n',
 'What can I do to improve my immune system?\n',
 'How is your relationship with your mother in law?\n',
 'How does one get Free PSN codes/Vita Codes?\n',
 'What is your review of osquery?\n',
 'How can I look smart and act smart?\n']

__Tokenization:__ a typical first step for an NLP task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many NLP tasks like tokenization, stemming or part-of-speech tagging.

In [4]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [5]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(sentence.lower()) for sentence in data]

In [6]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [7]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [8]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

# From gensim docs
# vw: This object essentially contains the mapping between words and embeddings.
# After training, it can be used directly to query those embeddings in various ways.

In [9]:
# now you can get word vectors !
model.get_vector('anything')

array([-3.0547047e+00,  5.9389508e-01,  9.4755733e-01,  1.0263950e+00,
        2.3875837e+00,  2.4389155e+00,  3.0452530e+00, -2.5023625e+00,
       -2.4952802e-01,  2.4183476e+00, -9.4922674e-01,  2.2128100e+00,
        5.0612721e+00,  7.2003037e-01,  1.7784556e+00,  3.4128910e-01,
       -1.5510134e+00, -3.0235434e-02,  7.2474974e-01, -2.6778028e+00,
       -3.1565616e+00,  4.8966059e-01, -2.4206111e+00, -3.3184838e-02,
       -1.0098811e+00, -3.1931221e+00, -1.1356534e+00, -1.9044657e+00,
        4.5909193e-05,  1.7669239e+00,  5.8958761e-02,  2.2360703e-01],
      dtype=float32)

In [10]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.947108268737793),
 ('cheese', 0.9390751123428345),
 ('sauce', 0.9281741976737976),
 ('beans', 0.9257345199584961),
 ('vodka', 0.9181455969810486),
 ('corn', 0.9178659915924072),
 ('noodles', 0.9153541326522827),
 ('honey', 0.9117775559425354),
 ('butter', 0.9112581610679626),
 ('fruit', 0.9081723093986511)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 
|
Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

After being downloaded for the first time (or if you manually delete it), the model is saved in the `~/gensim_data` or `%USER_PATH%/gensim_data` directory. This can be checked seting the return_path parameter to True.

In [11]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [12]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [13]:
words = model.index_to_key[:1000] 

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [14]:
# for each word, compute it's vector with model
word_vectors = np.asarray([model.get_vector(word) for word in words])

In [15]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is **P**rincipial **C**omponent **A**nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [16]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit_transform(word_vectors)

word_vectors_pca = word_vectors_pca - word_vectors_pca.mean(0)
word_vectors_pca = word_vectors_pca / word_vectors_pca.std(0)

In [17]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [18]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [19]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [20]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = TSNE().fit_transform(word_vectors)

In [21]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1154', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [23]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    vector = np.zeros([model.vector_size], dtype='float32')
    phrase_tok = tokenizer.tokenize(phrase.lower())
    # word_vectors = np.asarray([model.get_vector(word) for word in phrase_tok])
    result = []
    for word in phrase_tok:
        if word in model.key_to_index:
            result.append(model.get_vector(word))
    if len(result) == 0:
        return vector
    else:
        return np.asarray(result).mean(axis=0)

In [24]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))
assert np.array_equal(get_phrase_embedding("thisisgibberish"), np.zeros([model.vector_size], dtype='float32')), "corner case for all missing words should be handled as described in the function comments"

In [25]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.asarray([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [26]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [27]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [28]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1318', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [29]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [67]:
from sklearn.metrics.pairwise import cosine_similarity

def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query_emb = get_phrase_embedding(query)
    similarities = [cosine_similarity(query_emb.reshape(1, -1), d.reshape(1, -1)) for d in data_vectors]
    result = np.asarray([e[0][0] for e in similarities])
    result_ind = np.argpartition(-result, k)[:k]
    return [data[ind] for ind in result_ind]

In [68]:
results = find_nearest(query="How do i enter the matrix?", k=10)

In [69]:
# results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[2] == 'What can I do to save the world?\n'

How do I get to the dark web?
How do I win this?
What can I do to save the world?
How do I use the Greenify app?
What should I do to enter hollywood?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [70]:
find_nearest(query="How does Trump?", k=10)

['What does India think of Donald Trump?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think of India?\n',
 'What does Donald Trump think about Israel?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [71]:
find_nearest(query="Why don't i ask a question myself?", k=10)

["Why don't I get a girlfriend?\n",
 "Why don't I ever get a girl?\n",
 "Why can't I take a joke?\n",
 "Can I ask a girl out that I don't know?\n",
 "I don't have no question?\n",
 "Why don't I have a boyfriend?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why don't I have a girlfriend?\n"]

In [72]:
find_nearest(query="I love meat", k=10)

['What should I do if my dog eats chicken bones?\n',
 'How do I lose the weight even though I eat the food I love?\n',
 'How will I know if I ate chicken that was too raw?\n',
 'Why do I crave macaroni and cheese every night?\n',
 'Can I feed my dog human food like spaghetti and meatballs or potato wedges?\n',
 'How much spaghetti sauce do you need per person when making spaghetti?\n',
 'Am I missing out on much if I only eat vegetables and not fruits?\n',
 'I am 18 and hate all fruits and vegetables. I eat strictly meat, grains and Dairy. What will my health be if I continue to eat like this? (currently I am very fit)\n',
 'I love and care about animals. I love non-veg food. Am I a hypocrite?\n',
 'Is it possible I sneeze all the time because I ate spicy food?\n']

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize `find_nearest` with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.